In [5]:
import pandas as pd

# base.csvを読み込む
base_df = pd.read_csv('base2.csv')

# vector.csvを読み込む
vector_df = pd.read_csv('vector.csv')

# 必要なカラムをbase_dfから選択
new_df = base_df[['Title', 'Question_Num', 'url']].copy()

# Titleカラムの一文字目を取り出してdifficultiesカラムとして追加
new_df['difficulties'] = new_df['Title'].apply(lambda x: x[0] if isinstance(x, str) and x else None)

# Question_Numカラムの名前をqnumに変更
new_df.rename(columns={'Question_Num': 'qnum'}, inplace=True)

# vector_dfからclassifierカラムを追加
new_df['classifier'] = vector_df['classifier']

# classifierカラムを整数型に変換
new_df['classifier'] = new_df['classifier'].astype(int)

# カラムの順番を指定
new_df = new_df[['Title', 'difficulties', 'qnum', 'classifier', 'url']]

# Titleカラムの名前をtitleに変更
new_df.rename(columns={'Title': 'title'}, inplace=True)

# 新しいCSVファイルとして保存
new_df.to_csv('new_file.csv', index=False)


In [6]:
import pandas as pd
import sqlite3

# CSVファイルをDataFrameとして読み込む
base_df = pd.read_csv('base.csv')

# SQLiteデータベースファイル（base.db）を作成（または既存のものを開く）
conn = sqlite3.connect('base.db')

# DataFrameをデータベースに書き込む
base_df.to_sql('base_table', conn, if_exists='replace', index=False)

# 接続を閉じる
conn.close()


In [1]:
import sqlite3

# データベースへの接続
conn = sqlite3.connect('base.db')
cursor = conn.cursor()

# テーブルの作成
create_table_query = """CREATE TABLE IF NOT EXISTS questions (
                        Classifier INTEGER,
                        favorite_question INTEGER
                        )"""
cursor.execute(create_table_query)

# Classifierを1から15まで用意し、favorite_questionをすべて0に設定
for classifier in range(1, 16):
    cursor.execute("INSERT INTO questions (Classifier, favorite_question) VALUES (?, ?)", (classifier, 0))

# 変更をコミットして接続を閉じる
conn.commit()
cursor.close()
conn.close()


In [3]:
import sqlite3
import pandas as pd

# CSVファイルからデータの読み込み
base_df = pd.read_csv('base.csv')

# データベースへの接続
conn = sqlite3.connect('base.db')
cursor = conn.cursor()

# テーブルの作成
create_table_query = """CREATE TABLE IF NOT EXISTS titles (
                        Title TEXT,
                        WA INTEGER,
                        last_solve_date TEXT,
                        Attempts INTEGER
                        )"""
cursor.execute(create_table_query)

# base.csvからTitleカラムのデータを読み込み、他のカラムを追加
for index, row in base_df.iterrows():
    title = row['title']
    cursor.execute("INSERT INTO titles (Title, WA, last_solve_date, Attempts) VALUES (?, ?, ?, ?)", 
                   (title, 0, "2023/8/1", 0))

# 変更をコミットして接続を閉じる
conn.commit()
cursor.close()
conn.close()


In [6]:
import sqlite3
import pandas as pd

# CSVファイルからデータの読み込み
base_df = pd.read_csv('base.csv')

# データベースへの接続
conn = sqlite3.connect('base.db')
cursor = conn.cursor()

# カラムの存在をチェック
cursor.execute("PRAGMA table_info(titles)")
columns = cursor.fetchall()
if not any(column[1] == 'classifier' for column in columns):
    # titlesテーブルにclassifierカラムを追加
    cursor.execute("ALTER TABLE titles ADD COLUMN classifier INTEGER")

    # base.csvからclassifierカラムのデータを読み込み、titlesテーブルに追加
    for index, row in base_df.iterrows():
        classifier = row['classifier']
        cursor.execute("UPDATE titles SET classifier = ? WHERE Title = ?", 
                       (classifier, row['Title']))

# questionsテーブルにNot_goodカラムを追加
cursor.execute("PRAGMA table_info(questions)")
columns = cursor.fetchall()
if not any(column[1] == 'Not_good' for column in columns):
    cursor.execute("ALTER TABLE questions ADD COLUMN Not_good INTEGER DEFAULT 0")

# 変更をコミットして接続を閉じる
conn.commit()
cursor.close()
conn.close()


In [13]:
import sqlite3
import pandas as pd

# base.csvからtitleとclassifierカラムのみを読み込みます
base_df = pd.read_csv('base.csv', usecols=['title', 'classifier'])

# すべての行に対してWA、last_solve_date、Attemptsカラムを追加します
base_df['WA'] = 0
base_df['last_solve_date'] = '2023-08-01'
base_df['Attempts'] = 0

# データベースに接続します
conn = sqlite3.connect('base.db')
cursor = conn.cursor()

# 既存のテーブルがあれば削除します
cursor.execute("DROP TABLE IF EXISTS titles")

# 新しいテーブルを作成します
create_table_query = """CREATE TABLE titles (
                           title TEXT,
                           classifier INTEGER,
                           WA INTEGER,
                           last_solve_date TEXT,
                           Attempts INTEGER
                       )"""
cursor.execute(create_table_query)

# データをテーブルに挿入します
base_df.to_sql('titles', conn, if_exists='replace', index=False)

# 変更をコミットして接続を閉じます
conn.commit()
cursor.close()
conn.close()


In [14]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('base.db')
cursor = conn.cursor()

# Create a new temporary table with the same structure as 'base_table', plus the 'score' column
cursor.execute("""
CREATE TABLE base_table_temp AS
SELECT *, 0 AS score
FROM base_table
""")

# Drop the original table
cursor.execute("DROP TABLE base_table")

# Rename the temporary table to the original table name
cursor.execute("ALTER TABLE base_table_temp RENAME TO base_table")

# Commit the changes and close the connection
conn.commit()
cursor.close()
conn.close()
